**INSTITUTO NACIONAL DE PESQUISAS ESPACIAIS** 

Disciplina: Introdution to Data Science
    
Professores: Rafael Santos e Gilberto Queiroz
    
Acadêmica: Marcelly Homem Coelho
    
Contato: marcellyhc@gmail.com 

**Título:** Aplicação de Técnicas de Data Science no Desenvolvimento de um Sistema para Manutenção Aeronáutica Baseada em Condição 

**Descrição:** Este programa tem como objetivo analisar as mensagens de falha e as remoções dos sistemas das aeronaves.

In [26]:
# Importar as bibliotecas necessárias

import numpy as np
import pandas as pd
import seaborn as sns

import random

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
%matplotlib inline

# 1. Investigação Inicial da Estrutura e Conteúdo do Arquivo de Falha

In [27]:
# Criar um dataframe para entrada de dados de falha (arquivo do tipo .csv)

df_dataFailure = pd.read_csv('Data_Failure_Code.csv')   

In [28]:
# Exibir o cabeçalho do dataframe

df_dataFailure.head()

,Aircraft,Flight Phase,Date,Fault Text,Report Maintenance
0,2640,NaN,2006-05-14 16:19:00,FDE_Outhers02,NaN
1,2640,Enroute Cruise,2006-07-01 15:17:00,FDE_B_System3,RMA_A_System3
2,2640,Enroute Cruise,2006-07-01 15:17:00,FDE_C_System3,RMA_A_System3
3,2640,Enroute Cruise,2006-07-02 04:48:00,FDE_B_System3,RMA_A_System3
4,2640,Enroute Cruise,2006-07-02 04:48:00,FDE_C_System3,RMA_A_System3


In [29]:
# Verifica a dimensão do dataframe (qtd linhas, qtd colunas)

df_dataFailure.shape

(11, 5)

In [30]:
# Verificar o tipo de dado de cada coluna do dataframe

df_dataFailure.dtypes

Aircraft               int64
Flight Phase          object
Date                  object
Fault Text            object
Report Maintenance    object
dtype: object

In [31]:
# Determina quantas 'Flight Phase' diferentes há no dataframe

len(df_dataFailure['Aircraft'].unique())

1

# 2. Investigação Inicial da Estrutura e Conteúdo do Arquivo de Remoção

In [32]:
# Criar um dataframe para entrada de dados de remoção (arquivo do tipo .csv)

df_dataRemoval = pd.read_csv('Data_Removal_Code.csv')  

In [33]:
# Exibir do cabeçalho do dataframe

df_dataRemoval.head()

,Aircraft,Component,System,Date,Reason,Time Hours,Time Cycles
0,1140,MME_Component_A,System1,2006-05-29,3,118123,15961
1,1140,MME_Component_A,System1,2006-05-29,3,118123,15961
2,1140,MME_Component_B,System1,2006-05-29,3,1092,139
3,1140,MME_Component_B,System3,2006-06-24,3,312,37
4,1140,MME_Component_B,System3,2006-07-10,3,118698,16028


In [34]:
# Verifica a dimensão do dataframe (qtd linhas, qtd colunas)

df_dataFailure.shape

(11, 5)

In [35]:
# Verificar o tipo de dado de cada coluna do dataframe

df_dataFailure.dtypes

Aircraft               int64
Flight Phase          object
Date                  object
Fault Text            object
Report Maintenance    object
dtype: object